In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]=os.environ.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="anthropic_iterative_search"

In [2]:
def _format_docs(docs):
    result = "\n".join(
        [
            f'<item index="{i+1}">\n<page_content>\n{r}\n</page_content>\n</item>'
            for i, r in enumerate(docs)
        ]
    )
    return result


def format_agent_scratchpad(intermediate_steps):
    thoughts = ""
    for action, observation in intermediate_steps:
        thoughts += action.log
        thoughts += "</search_query>" + _format_docs(observation)
    return thoughts

In [3]:
retrieval_prompt = """{retriever_description} Before beginning to research the user's question, \n
first think for a moment inside <scratchpad> tags about what information is necessary for a well-informed answer. \n
If the user's question is complex, you may need to decompose the query into multiple subqueries and execute them individually. \n
Sometimes the search engine will return empty search results, or the search results may not contain the information you need. \n
In such cases, feel free to try again with a different query. 

After each call to the Search Engine Tool, reflect briefly inside <search_quality></search_quality> \n
tags about whether you now have enough information to answer, or whether more information is needed. \n
If you have all the relevant information, write it in <information></information> tags, \n
WITHOUT actually answering the question. Otherwise, issue a new search.

Here is the user's question: <question>{query}</question> \n
Remind yourself to make short queries in your scratchpad as you plan out your strategy.""" 

answer_prompt = """Here is a user query: <query>{query}</query>. \n
Here is some relevant information: \n<information>{information}</information>. \n
Please answer the question using the relevant information."""

In [4]:
from langchain.retrievers import WikipediaRetriever
from langchain.tools import tool

# This is used to tell the model how to best use the retriever.

retriever_description = """You will be asked a question by a human user. \n
You have access to the following tool to help answer the question. \n

<tool_description> \n

Search Engine Tool \n

* The search engine will exclusively search over Wikipedia for pages similar to your query. \n
It returns for each page its title and full page content. \n
Use this tool if you want to get up-to-date and comprehensive information on a topic to help answer queries. \n

Queries should be as atomic as possible -- they only need to address one part of the user's question. \n
For example, if the user's query is "what is the color of a basketball?", your search query should be "basketball". \n
Here's another example: if the user's question is "Who created the first neural network?", \n
your first query should be "neural network". As you can see, these queries are quite short. \n
Think keywords, not phrases. \n

* At any time, you can make a call to the search engine using the following syntax: \n

<search_query>query_word</search_query>. \n

* You'll then get results back in <search_result> tags.\n

</tool_description>""" 

retriever = WikipediaRetriever()

# This should be the same as the function name below
RETRIEVER_TOOL_NAME = "search"


@tool
def search(query):
    """Search with the retriever."""
    return retriever.invoke(query)

In [5]:
import re

from langchain_core.agents import AgentAction, AgentFinish


def extract_between_tags(tag: str, string: str, strip: bool = True) -> str:
    ext_list = re.findall(f"<{tag}\s?>(.+?)</{tag}\s?>", string, re.DOTALL)
    if strip:
        ext_list = [e.strip() for e in ext_list]
    if ext_list:
        if len(ext_list) != 1:
            raise ValueError
        # Only return the first one
        return ext_list[0]


def parse_output(outputs):
    partial_completion = outputs["partial_completion"]
    steps = outputs["intermediate_steps"]
    search_query = extract_between_tags(
        "search_query", partial_completion + "</search_query>"
    )
    if search_query is None:
        docs = []
        str_output = ""
        for action, observation in steps:
            docs.extend(observation)
            str_output += action.log
            str_output += "</search_query>" + _format_docs(observation)
        str_output += partial_completion
        return AgentFinish({"docs": docs, "output": str_output}, log=partial_completion)
    else:
        return AgentAction(
            tool="search", tool_input=search_query, log=partial_completion
        )

In [6]:
from langchain.agents import AgentExecutor
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

prompt = ChatPromptTemplate.from_messages(
    [
        ("user", retrieval_prompt),
        ("ai", "{agent_scratchpad}"),
    ]
)
prompt = prompt.partial(retriever_description=retriever_description)

model = ChatOpenAI(
    model="gpt-4o-mini", temperature=0, max_tokens=1000
)

chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_agent_scratchpad(x["intermediate_steps"])
    )
    | prompt
    | model.bind(stop=["</search_query>"])
    | StrOutputParser()
)

agent_chain = (
    RunnableParallel(
        {
            "partial_completion": chain,
            "intermediate_steps": lambda x: x["intermediate_steps"],
        }
    )
    | parse_output
)

executor = AgentExecutor(agent=agent_chain, tools=[search], verbose=True)

c:\Code\Github\LangChain-Basics\venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import ConfigurableField

prompt = ChatPromptTemplate.from_template(answer_prompt)

model = ChatOpenAI(
    model="gpt-4o-mini", temperature=0, max_tokens=1000
)

chain = (
    {"query": lambda x: x["query"], "information": executor | (lambda x: x["output"])}
    | prompt
    | model
    | StrOutputParser()
)

# Add typing for the inputs to be used in the playground


class Inputs(BaseModel):
    query: str


chain = chain.with_types(input_type=Inputs)

chain = chain.configurable_alternatives(
    ConfigurableField(id="chain"),
    default_key="response",
    # This adds a new option, with name `openai` that is equal to `ChatOpenAI()`
    retrieve=executor,
)

In [8]:
query = (
        "Which movie came out first: Oppenheimer, or "
        "Are You There God It's Me Margaret?"
    )

In [9]:
result = chain.invoke({"query":query})



> Entering new AgentExecutor chain...
<scratchpad>To determine which movie came out first, I need to find the release dates for both "Oppenheimer" and "Are You There God It's Me Margaret." I will start by searching for each movie individually to get their respective release dates.</scratchpad> 

<search_query>Oppenheimer[Document(page_content='J. Robert Oppenheimer (born Julius Robert Oppenheimer;  OP-ən-hy-mər; April 22, 1904 – February 18, 1967) was an American theoretical physicist who served as the director of the Manhattan Project\'s Los Alamos Laboratory during World War II. He is often called the "father of the atomic bomb" for his role in overseeing the development of the first nuclear weapons.\nBorn in New York City, Oppenheimer obtained a degree in chemistry from Harvard University in 1925 and a doctorate in physics from the University of Göttingen in Germany in 1927, studying under Max Born. After research at other institutions, he joined the physics faculty at the Univers